In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DMT_PRD")
dbutils.widgets.text("target_table", "a_finance_bill_sll_detail")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text("dependency_table",
     "VCM_DWH_PRD.D_PRODUCT, VCM_DWH_PRD.D_PROMOTION, VCM_DWH_PRD.D_STORE, VCM_DWH_PRD.D_WIN_MEMBERSHIP, VCM_DWH_PRD.F_COGS, VCM_DWH_PRD.F_SALE_PROD, VCM_DWH_PRD.F_SALEOUT_BILL_HEADER, VCM_STG_INF.POS_WCM_SALEOUT_PAYMENT"
)

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
catalog_name = settings[environment]['catalog_name']


In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
%run ../common/common_etl_load

In [0]:
spark.sql(f"""
CREATE  TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dmt.a_finance_bill_sll_detail
(
  dataset STRING,
  business_unit STRING,
  calday DATE,
  transaction_index STRING,
  bill_type STRING,
  transaction_type STRING,
  transaction_sale STRING,
  transaction_no STRING,
  product_id STRING,
  product_name STRING,
  mch2_id STRING,
  mch2_name STRING,
  mch3_name STRING,
  mch4_name STRING,
  store_id STRING,
  store_name STRING,
  data_line_type STRING,
  promotion STRING,
  payment_method STRING,
  payment_method_description STRING,
  tender_value FLOAT,
  list_voucher_code STRING,
  released_voucher_store_id BIGINT,
  pos_number STRING,
  so_the_vinid STRING,
  base_unit STRING,
  sales_unit STRING,
  base_sale_qty FLOAT,
  sale_qty FLOAT,
  sale_amount FLOAT,
  tax_amount FLOAT,
  revenue FLOAT,
  cogs FLOAT,
  front_margin FLOAT,
  frt_margin_prcnt FLOAT,
  revenue_per_baseunit FLOAT,
  total_prd_of_bill BIGINT,
  total_bill_of_store_sku BIGINT,
  vat FLOAT,
  total_pro_excl_vat FLOAT,
  zbbprtven FLOAT,
  bonus_buy_id STRING,
  wcm_promotion FLOAT,
  online_offline STRING,
  dautu_wcm STRING,
  total_bill_amount FLOAT,
  sale_mix_fresh_rextile_homeline FLOAT,
  mbs_id STRING)
  TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW temp_a_finance_bill_sll_detail AS
WITH
  return_bill AS (
    SELECT DISTINCT
      a.receipt_no transaction_no,
      a.ref_01_ref_bill
    FROM
      {catalog_name}.udp_wcm_gold_vcm_dwh.f_saleout_bill_header a
      INNER JOIN (SELECT DISTINCT transaction_no FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_finance_bill_sll) b ON a.ref_01_ref_bill = LTRIM('0', b.transaction_no)
    WHERE a.calday BETWEEN DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 2 AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1
  ),
  f_sale_prod_tmp AS (
    SELECT
      a.hash_id, a.calday, a.base_uom, a.sales_unit, a.transaction_index, a.transaction_id, a.product_id, a.business_unit, a.store_id, a.promotion, a.transaction_no, a.transaction_type, a.time_of_day, a.sale_time, a.credit_card_id, a.payment_method, a.department, a.pos_number, a.offer, a.discount_type, a.discount_type_group, a.recordmode, a.return_qty1, a.return_qty2, a.return_value, a.promotion_qty, a.base_sale_qty, a.receipt_count, a.sale_in_cost_value, a.promotion_sale, a.sale_qty, a.sale_amount, a.promotion_sale_l, a.return_sale_l, a.discount_item_no, a.discount_value, a.tax_amount, a.tender_value, a.vinid_card_no, a.is_sll,
      LTRIM('0',transaction_no) transaction_sale,
      'BILL BÁN' bill_type,
      CASE
        WHEN (IFNULL(a.base_sale_qty, 0) <> 0 AND product_id IS NOT NULL) THEN 'SALE_LINE_ITEM'
        WHEN IFNULL(a.tender_value, 0) <> 0 THEN 'PAYMENT'
        WHEN discount_type = 'ZPRM' THEN 'PROMOTION'
        WHEN discount_type IN ('ZRTA', 'ZCRT') THEN 'TAX_AMT_VIADISCOUNT'
        ELSE 'OTHER'
      END data_line_type
    FROM
      {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod a
      INNER JOIN (SELECT DISTINCT transaction_no FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_finance_bill_sll WHERE dataset <> 'NA') b USING (transaction_no)
    WHERE a.calday BETWEEN DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 2 AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1
    UNION ALL
    SELECT
       a.hash_id, a.calday, a.base_uom, a.sales_unit, a.transaction_index, a.transaction_id, a.product_id, a.business_unit, a.store_id, a.promotion, a.transaction_no, a.transaction_type, a.time_of_day, a.sale_time, a.credit_card_id, a.payment_method, a.department, a.pos_number, a.offer, a.discount_type, a.discount_type_group, a.recordmode, a.return_qty1, a.return_qty2, a.return_value, a.promotion_qty, a.base_sale_qty, a.receipt_count, a.sale_in_cost_value, a.promotion_sale, a.sale_qty, a.sale_amount, a.promotion_sale_l, a.return_sale_l, a.discount_item_no, a.discount_value, a.tax_amount, a.tender_value, a.vinid_card_no, a.is_sll,
       b.ref_01_ref_bill transaction_sale,
      'BILL TRẢ HÀNG' bill_type,
      CASE
        WHEN (IFNULL(a.base_sale_qty, 0) <> 0 AND product_id IS NOT NULL) THEN 'SALE_LINE_ITEM'
        WHEN IFNULL(a.tender_value, 0) <> 0 THEN 'PAYMENT'
        WHEN discount_type = 'ZPRM' THEN 'PROMOTION'
        WHEN discount_type IN ('ZRTA', 'ZCRT') THEN 'TAX_AMT_VIADISCOUNT'
        ELSE 'OTHER'
      END data_line_type
    FROM
      {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod a
      INNER JOIN return_bill b ON LTRIM('0',a.transaction_no) = b.transaction_no
    WHERE a.calday BETWEEN DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 2 AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1
  ),
  f_sale_prod AS (
    SELECT a.* EXCEPT(tax_amount),
           a.tax_amount - IF(a.data_line_type = 'SALE_LINE_ITEM',IFNULL(b.discount_value,0),0) tax_amount
    FROM  (SELECT * FROM f_sale_prod_tmp WHERE data_line_type NOT IN ('OTHER','TAX_AMT_VIADISCOUNT')) a
    LEFT JOIN
          (SELECT
            transaction_no,
            product_id,
            sales_unit,
            SUM(discount_value) discount_value
          FROM f_sale_prod_tmp WHERE data_line_type = 'TAX_AMT_VIADISCOUNT'
          GROUP BY 1,2,3
          ) b
          USING (transaction_no,product_id,sales_unit)
  ),
  cogs AS (
    SELECT
      calday,
      store_id,
      product_id,
      cogs_per_unit,
      cost,
      quantity
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_cogs
    WHERE calday BETWEEN DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 2 AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1
  ),
  voucher_info AS (
    SELECT
      CONCAT('00000', receipt_no) transaction_no,
      tender_type payment_method,
      concat_ws(';', collect_list(ref_no)) list_voucher_code
    FROM {catalog_name}.udp_wcm_silver_eventhub.pos_wcm_saleout_payment
    WHERE
      calday BETWEEN DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 2 AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1
      AND tender_type IN ('E-ZIVO','ZFIV','ZIVO','ZPVO','ZUFV','ZUVO','ZVIV')
    GROUP BY 1, 2
  ),
  promo_tyledautu AS (
    SELECT DISTINCT
      bonus_buy_id,
      store_id,
      zbbprtven
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_promotion
    WHERE promo_end_date >= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 2 AND zbbprtven < 100
  ),
  bill_wcm_promo AS (
    SELECT DISTINCT
      transaction_no,
      a.store_id,
      bonus_buy_id,
      zbbprtven
    FROM
      {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod a
      INNER JOIN promo_tyledautu b ON a.promotion = b.bonus_buy_id AND a.store_id = b.store_id
       LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product c ON a.product_id = c.product_id
    WHERE
      a.calday BETWEEN DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 2 AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1
      AND c.mch2_id IN ('102','202')
      AND c.mch3_id NOT IN ('10204','10205')
  ),
  vat_per_sku AS (
    SELECT DISTINCT
      transaction_no,
      product_id,
      sales_unit,
     ROUND(sum(tax_amount) / sum(sale_amount - tax_amount), 3) AS vat
      FROM f_sale_prod
      WHERE data_line_type = 'SALE_LINE_ITEM'
      GROUP BY 1,2,3
  ),
  kdol_sale AS (
    SELECT
     DISTINCT LTRIM('0',transaction_no) transaction_no
    FROM
      {catalog_name}.udp_wcm_gold_vcm_db_online.f_sale_prod_kdol_flat_non_dichoho
    WHERE  calday BETWEEN '2023-01-01' AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1
  ),
  final AS (
    SELECT
      IF(data_line_type <> 'SALE_LINE_ITEM','NA',IFNULL(a1.dataset,'NA')) dataset,
      a.business_unit,
      a.calday,
      a.transaction_index,
      a.bill_type,
      a.transaction_type,
      a.transaction_sale,
      a.transaction_no,
      a.product_id,
      prd.product_name,
      prd.mch2_id,
      prd.mch2_desc mch2_name,
      prd.mch3_desc mch3_name,
      prd.mch4_desc mch4_name,
      a.store_id,
      st.store_name,
      a.data_line_type,
      a.promotion,
      a.payment_method,
      b.description AS payment_method_description,
      a.tender_value,
      v.list_voucher_code,
      NULL released_voucher_store_id,
      a.pos_number,
      a.vinid_card_no so_the_vinid,
      a.base_uom base_unit,
      a.sales_unit,
      a.base_sale_qty,
      a.sale_qty,
      a.sale_amount,
      a.tax_amount,
      a.sale_amount - a.tax_amount AS revenue,
      c.cogs_per_unit * a.base_sale_qty cogs,
      (a.sale_amount - a.tax_amount) - (c.cogs_per_unit * a.base_sale_qty) front_margin,
     try_divide(
    (a.sale_amount - a.tax_amount) - (c.cogs_per_unit * a.base_sale_qty),
    a.sale_amount - a.tax_amount
) AS frt_margin_prcnt,
try_divide(
    a.sale_amount - a.tax_amount,
    a.base_sale_qty
) AS revenue_per_baseunit,
      a1.total_prd_of_bill,
      a1.total_bill_of_store_sku,
      a1.is_voucher,
      a1.is_promo_wcm,
      a1.is_vinid,
      e.vat,
      (-1) * IF(a.data_line_type = 'PROMOTION',discount_value / (1+IFNULL(e.vat,0)),0) total_pro_excl_vat,
      d.zbbprtven, d.bonus_buy_id,
      (-1) * IF(a.data_line_type = 'PROMOTION',discount_value / (1+IFNULL(e.vat,0)),0)*IFNULL(1-D.zbbprtven/100, 0) wcm_promotion,
      IF(f.transaction_no IS NULL, 'OFFLINE', 'ONLINE') online_offline
    FROM
      f_sale_prod a
      LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dmt.a_finance_bill_sll  a1 ON LTRIM('0',a.transaction_sale) = LTRIM('0',a1.transaction_no) AND a.product_id = a1.product_id
      LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product prd ON a.product_id = prd.product_id
      LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st ON a.store_id = st.store_id
      LEFT JOIN {catalog_name}.udp_wcm_silver_dim.payment_method b ON a.payment_method = b.payment_method
      LEFT JOIN cogs c ON a.calday = c.calday AND a.store_id = c.store_id AND a.product_id = c.product_id
      LEFT JOIN bill_wcm_promo d ON a.store_id = d.store_id AND a.transaction_no = d.transaction_no AND a.promotion = d.bonus_buy_id
      LEFT JOIN voucher_info v ON a.transaction_no = v.transaction_no AND a.payment_method = v.payment_method
      LEFT JOIN vat_per_sku e ON a.transaction_no = e.transaction_no AND a.product_id = e.product_id AND a.sales_unit = e.sales_unit
      LEFT JOIN kdol_sale f ON LTRIM('0',a.transaction_sale) = LTRIM('0',f.transaction_no)
    WHERE data_line_type <> 'OTHER'
  )

  SELECT
    dataset,
    a.* EXCEPT(is_voucher, is_vinid, is_promo_wcm,dataset),
    CONCAT('IS_VOUCHER-', is_voucher,'; ','IS_VINID-',a.is_vinid,'; ','IS_PROMO_WCM-',is_promo_wcm) dautu_wcm,
    SUM(sale_amount) OVER(PARTITION BY transaction_no) total_bill_amount,
   try_divide(
  SUM(CASE WHEN mch2_id IN ('101','201','203') THEN sale_amount ELSE 0 END)
    OVER (PARTITION BY transaction_no),
  SUM(sale_amount) OVER (PARTITION BY transaction_no)
) AS sale_mix_fresh_rextile_homeline,
    c.capilary_customer_id mbs_id
  FROM
    final a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.f_saleout_bill_header  b ON LTRIM(a.transaction_sale) = b.receipt_no
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_win_membership c ON b.mobile_number = c.mobile_number;
""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_finance_bill_sll_detail
WHERE calday BETWEEN DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 2 AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dmt.a_finance_bill_sll_detail
SELECT * FROM temp_a_finance_bill_sll_detail
""")

In [0]:
%run "../common/common_etl_update"

In [0]:
spark.sql("DROP VIEW IF EXISTS temp_a_finance_bill_sll_detail")